# Training loop

In questo notebook analizzeremo i vari passaggi necessari per il training di un modello.

In breve, un ciclo di addestramento si caratterizza di 3 fasi:

 - dataloader
 - training step
 - validation step
 
Sebbene la maggior parte delle librerie per ML fornisca delle utils già predisposte per eseguire questi passaggi, avere una panoramica delle varie fasi diventa fondamentale nel caso ci sia bisogno di **modifiche customizzate** e per fini di **debugging**.  

## Dataloader

Il `Dataloader` è l'oggetto responsabile di leggere opportunamente i dati, accoppiare opportunamente input ed output, trasferirli dal disco in memoria sotto forma di **batch** e renderli disponibili per il processo di training.

La libreria fastai mette a disposizione diversi dataloader che implementano le operazioni fondamentali a seconda del task di learning in esame. Queste implementazioni sono tipicamente già sufficienti per use-case più comuni come il nostro, per questo motivo potremo sfruttare direttamente la classe `SegmentationDataLoaders`.

In particolare, potremo usare l'implementazione del metodo `SegmentationDataLoaders.from_label_func()` e fornire semplicemente i blocchi necessari per il suo funzionamento:

 - *path*: percoso dove sono i dati
 - *fnames*: lista percorsi di ciascun file di input
 - *label_func*: funzione che permette di associare a ciascun input la relativa annotazione
 - *bs*: batch size
 - *splitter*: funzione che restituisce `True` se il file appartiene al validation set, `False` altrimenti
 - *item_tfms*: transformazioni da applicare sui singoli file di input, prima del raggruppamento in batch
 - *batch_tfms*: trasformazioni da applicare a ciascun batch
 - *device*: dove caricare il batch risultante; "cuda" per usare GPU 




###  Percosi immagini di train e validation

In [1]:
from fastai.vision.all import *

data_path = Path('../fluocells_data/all_images')

# read train/valid/test split dataframe
split_df = pd.read_csv(data_path.parent / 'split_df.csv')
trainval_fnames = [data_path / 'images' / fn
                   for fn in split_df.query("split=='train' or split=='valid'").filename]

trainval_fnames

[Path('../fluocells_data/all_images/images/Mar26bS2C1R2_LHl_200x_y.png'),
 Path('../fluocells_data/all_images/images/Mar26bS1C2R1_VLPAGr_200x_y.png'),
 Path('../fluocells_data/all_images/images/Mar26bS1C1R4_VLPAGl_200x_y.png'),
 Path('../fluocells_data/all_images/images/MAR38S1C3R1_DMR_20_o.png'),
 Path('../fluocells_data/all_images/images/Mar19bS1C4R3_DMl_200x_y.png'),
 Path('../fluocells_data/all_images/images/Mar20bS2C1R1_LHl_200x_y.png'),
 Path('../fluocells_data/all_images/images/Mar24bS1C2R3_LHr_200x_y.png'),
 Path('../fluocells_data/all_images/images/Mar26bS2C2R1_LHr_200x_y.png'),
 Path('../fluocells_data/all_images/images/Mar21bS1C1R3_VLPAGr_200x_y.png'),
 Path('../fluocells_data/all_images/images/Mar21bS1C4R2_LHl_200x_y.png'),
 Path('../fluocells_data/all_images/images/Mar37S1C2R1_DMr_200x_o.png'),
 Path('../fluocells_data/all_images/images/Mar22bS2C1R1_LHl_200x_y.png'),
 Path('../fluocells_data/all_images/images/Mar23bS1C2R4_VLPAGl_200x_y.png'),
 Path('../fluocells_data/all_i

### label_func

In [2]:
def label_func(p):
    return Path(str(p).replace('images', 'masks'))

In [3]:
print('Esempio:\n')

print(f'Input: {trainval_fnames[0]}')
print(f'Output: {label_func(trainval_fnames[0])}')

Esempio:

Input: ../fluocells_data/all_images/images/Mar26bS2C1R2_LHl_200x_y.png
Output: ../fluocells_data/all_masks/masks/Mar26bS2C1R2_LHl_200x_y.png


### Splitter

In [4]:
sample_img_names = ['Mar32bS2C2R2_DMl_200x_y.png', 'Mar27bS1C2R1_LHr_200x_y.png',
                    'Mar26bS2C1R2_LHr_200x_y.png', 'Mar41S3C1R1_DMr_200x_o.png']

print('split dataframe:\n', split_df.loc[split_df.filename.isin(sample_img_names)])

def is_valid(p):
    return split_df.loc[split_df.filename == p.name, 'split'].values[0] == 'valid'

print('splitter:\n')
for fn in sample_img_names:
    print(data_path / 'images' / fn, '\tvalidation set:', is_valid(data_path / 'images' / fn))

split dataframe:
      img_id                     filename  split
81       81   Mar41S3C1R1_DMr_200x_o.png  train
194     194  Mar32bS2C2R2_DMl_200x_y.png  train
196     196  Mar26bS2C1R2_LHr_200x_y.png  valid
248     248  Mar27bS1C2R1_LHr_200x_y.png  valid
splitter:

../fluocells_data/all_images/images/Mar32bS2C2R2_DMl_200x_y.png 	validation set: False
../fluocells_data/all_images/images/Mar27bS1C2R1_LHr_200x_y.png 	validation set: True
../fluocells_data/all_images/images/Mar26bS2C1R2_LHr_200x_y.png 	validation set: True
../fluocells_data/all_images/images/Mar41S3C1R1_DMr_200x_o.png 	validation set: False


### Transforms

### Dataloader 

In [6]:
# augmentation
tfms = [
    IntToFloatTensor(div_mask=255.),  # need masks in [0, 1] format
    RandomCrop(512),
    *aug_transforms(
        size=224, # resize
        max_lighting=0.1, p_lighting=0.5, # variazione luminosità
        min_zoom=0.9, max_zoom=1.1, # zoom
        max_warp=0, # distorsione
        max_rotate=15.0 # rotazione
    )
]


# splitter
splitter = FuncSplitter(lambda p: is_valid(p))
# alternative: random splitter
# RandomSplitter(valid_pct=0.3, seed=42),

# dataloader
dls = SegmentationDataLoaders.from_label_func(
    data_path, fnames=trainval_fnames, label_func=label_func,
    bs=2,
    splitter=splitter,
    #     item_tfms=pre_tfms,
    batch_tfms=tfms,
    device='cuda'
)

# test
x, y = dls.one_batch()
print(x.shape, y.shape)
x.min(), x.max(), y.min(), y.max()

/home/luca/anaconda3/envs/fx/lib/python3.9/site-packages/torch/_tensor.py:1051: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ret = func(*args, **kwargs)


torch.Size([2, 3, 224, 224]) torch.Size([2, 224, 224])


(TensorImage(9.8221e-08, device='cuda:0'),
 TensorImage(0.9783, device='cuda:0'),
 TensorMask(0, device='cuda:0'),
 TensorMask(1, device='cuda:0'))